# Implementación Naive Bayes para Seguimiento Docente

## Objetivo

Este notebook es una guía práctica que muestra, paso a paso, cómo implementar un clasificador **Naive Bayes** utilizando Python y la librería Scikit-learn. El objetivo es predecir la valoración del desempeño de un docente (`Valoracion`) basándonos en las características de los cursos que imparte, utilizando los datos de seguimiento proporcionados.

## Paso 1: Importar las Librerías Necesarias

Primero, importamos todas las herramientas que vamos a necesitar.

In [2]:
## Paso 2: Importar las Librerías Necesarias

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import warnings

# Ocultar advertencias para una salida más limpia
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente.")

✅ Librerías importadas correctamente.


## Paso 2: Cargar y Preparar los Datos
 
Cargamos los datos desde un archivo (`.csv`, `.xlsx`).

In [3]:
## Paso 3: Carga y Exploración Inicial de Datos

# Cargar el archivo CSV
try:
    df = pd.read_csv('datos_IA.csv', sep=';')
    print("✅ Archivo 'datos_IA.csv' cargado exitosamente.")
except FileNotFoundError:
    print("❌ Error: El archivo 'datos_IA.csv' no se encuentra.")
    # Usar un DataFrame de ejemplo si el archivo no está
    df = pd.DataFrame() 

if not df.empty:
    # Mostrar las primeras filas y la información del DataFrame
    print("\nPrimeras 5 filas del DataFrame:")
    display(df.head())
    print("\nInformación del DataFrame (tipos de datos y valores no nulos):")
    df.info()

✅ Archivo 'datos_IA.csv' cargado exitosamente.

Primeras 5 filas del DataFrame:


,Deporte o Servicio,Nivel,Edad,Género,Etnia,Tipo de Poblacion,Escenario,Municipio,Diversidad
0,CENTRO DE INICIACIÓN Y FORMACIÓN DEPORTIVA,INSCRIPCIÓN NIÑOS (7 - 8 AÑOS),9,FEMENINO,Mestiza,Ninguna de las anteriores,ALTO DE LOS SUEÑOS,ENVIGADO,Heterosexual
1,CENTRO DE INICIACIÓN Y FORMACIÓN DEPORTIVA,INSCRIPCIÓN NIÑOS (5 - 6 AÑOS),7,FEMENINO,Mestiza,Ninguna de las anteriores,ALTO DE LOS SUEÑOS,ENVIGADO,Heterosexual
2,CENTRO DE INICIACIÓN Y FORMACIÓN DEPORTIVA,INSCRIPCIÓN NIÑOS (6 - 7 AÑOS),10,MASCULINO,Otro,Ninguna de las anteriores,ALTO DE LOS SUEÑOS,ENVIGADO,Heterosexual
3,CENTRO DE INICIACIÓN Y FORMACIÓN DEPORTIVA,INSCRIPCIÓN NIÑOS (6 - 7 AÑOS),9,MASCULINO,Mestiza,Ninguna de las anteriores,ALTO DE LOS SUEÑOS,ENVIGADO,Heterosexual
4,CENTRO DE INICIACIÓN Y FORMACIÓN DEPORTIVA,INSCRIPCIÓN NIÑOS (6 - 7 AÑOS),9,FEMENINO,Mestiza,Ninguna de las anteriores,ALTO DE LOS SUEÑOS,ENVIGADO,Heterosexual



Información del DataFrame (tipos de datos y valores no nulos):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71709 entries, 0 to 71708
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Deporte o Servicio  71709 non-null  object
 1   Nivel               71709 non-null  object
 2   Edad                71709 non-null  int64 
 3   Género              71709 non-null  object
 4   Etnia               71709 non-null  object
 5   Tipo de Poblacion   71709 non-null  object
 6   Escenario           71709 non-null  object
 7   Municipio           71709 non-null  object
 8   Diversidad          71709 non-null  object
dtypes: int64(1), object(8)
memory usage: 4.9+ MB


## Paso 3: Preprocesamiento de los Datos

Los modelos de Machine Learning no entienden texto. Por lo tanto, debemos convertir nuestras características categóricas (`Tipo`, `Tipo_Curso`, `Plantilla`) en representaciones numéricas. Este proceso se llama **codificación**.

1.  **Separar Características (X) y Variable Objetivo (y)**: Aislamos las columnas que usaremos para predecir (X) de la columna que queremos predecir (y).
2.  **Codificar Características**: Usaremos `OrdinalEncoder` para asignar un número entero único a cada categoría de texto.

In [4]:
## Paso 4: Preparación de Datos: Selección de Variables

# Definir la variable objetivo (Y) y las variables predictoras (X)
TARGET_VARIABLE = 'Deporte o Servicio'

# Eliminamos la variable objetivo de las características
features = [col for col in df.columns if col != TARGET_VARIABLE]

X = df[features]
Y = df[TARGET_VARIABLE]

print(f"Variable Objetivo (Y): {TARGET_VARIABLE}")
print(f"Características (X): {features}")
print(f"Tamaño de X: {X.shape}")
print(f"Tamaño de Y: {Y.shape}")

Variable Objetivo (Y): Deporte o Servicio
Características (X): ['Nivel', 'Edad', 'Género', 'Etnia', 'Tipo de Poblacion', 'Escenario', 'Municipio', 'Diversidad']
Tamaño de X: (71709, 8)
Tamaño de Y: (71709,)


## Paso 4: Dividir los Datos en Conjuntos de Entrenamiento y Prueba

Es una práctica fundamental en Machine Learning. Entrenamos el modelo con la mayoría de los datos (conjunto de entrenamiento) y luego evaluamos su rendimiento en un conjunto de datos que nunca ha visto (conjunto de prueba). Esto nos da una medida de su capacidad para generalizar.

In [5]:
## Paso 5: Preprocesamiento: Imputación y Codificación Ordinal

# Naive Bayes Categórico requiere que las variables de entrada sean categóricas
# y codificadas como números enteros.

# 1. Identificar variables categóricas
categorical_features = X.columns.tolist()

# 2. Imputar (rellenar) valores faltantes
# Usamos el modo (el valor más frecuente) para rellenar los valores nulos/vacíos.
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
print("✅ Valores faltantes imputados con la moda.")

# 3. Codificación Ordinal para convertir las cadenas de texto a números enteros
# Este es un requisito para el Categorical Naive Bayes en scikit-learn.
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_encoded = encoder.fit_transform(X_imputed)

# 4. También codificamos la variable objetivo (si es categórica, lo es en este caso)
# Aunque CategoricalNB puede tomar cadenas en Y, es una buena práctica codificar.
Y_encoded = pd.Series(encoder.fit_transform(Y.values.reshape(-1, 1)).flatten())
target_classes = encoder.categories_[0].tolist() # Guardar el mapeo de las clases

print("\nPrimeras 5 filas de X después de la codificación:")
display(pd.DataFrame(X_encoded, columns=X.columns).head())

✅ Valores faltantes imputados con la moda.

Primeras 5 filas de X después de la codificación:


,Nivel,Edad,Género,Etnia,Tipo de Poblacion,Escenario,Municipio,Diversidad
0,190.0,8.0,0.0,3.0,4.0,2.0,12.0,2.0
1,188.0,6.0,0.0,3.0,4.0,2.0,12.0,2.0
2,189.0,9.0,1.0,9.0,4.0,2.0,12.0,2.0
3,189.0,8.0,1.0,3.0,4.0,2.0,12.0,2.0
4,189.0,8.0,0.0,3.0,4.0,2.0,12.0,2.0


## Paso 5: Entrenar el Modelo Naive Bayes

Ahora llega el momento de crear y entrenar nuestro clasificador. Como nuestras características son categóricas, el modelo `CategoricalNB` de Scikit-learn es el más apropiado.

In [6]:
## Paso 6: División de Datos

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_encoded, 
    Y_encoded, 
    test_size=0.2, 
    random_state=42,
    stratify=Y_encoded # Usar stratify para asegurar una distribución similar de la variable objetivo
)

print(f"Datos de Entrenamiento (X_train): {X_train.shape}")
print(f"Datos de Prueba (X_test): {X_test.shape}")

Datos de Entrenamiento (X_train): (57367, 8)
Datos de Prueba (X_test): (14342, 8)


## Paso 6: Evaluar el Modelo

Una vez entrenado, usamos el modelo para hacer predicciones sobre el conjunto de prueba y comparamos esas predicciones con los resultados reales para medir su rendimiento.

In [7]:
## Paso 7: Entrenamiento del Modelo Naive Bayes Categórico

# Inicializar el clasificador Naive Bayes Categórico
cnb = CategoricalNB()

# Entrenar el modelo
cnb.fit(X_train, Y_train)

print("✅ Modelo Categorical Naive Bayes entrenado exitosamente.")

✅ Modelo Categorical Naive Bayes entrenado exitosamente.


## Paso 7: Realizar una Predicción para un Nuevo Caso

Esta es la aplicación final del modelo: usarlo para obtener información sobre un caso nuevo. Crearemos un nuevo curso y le pediremos al modelo que prediga su valoración de desempeño.

In [8]:
## Paso 8: Predicción y Evaluación del Modelo

# Realizar predicciones sobre el conjunto de prueba
Y_pred = cnb.predict(X_test)

# 1. Calcular la Precisión (Accuracy)
accuracy = accuracy_score(Y_test, Y_pred)

print(f"\n✨ Precisión del Modelo (Accuracy): {accuracy:.4f}")

# 2. Generar el Reporte de Clasificación Detallado
# Nota: La decodificación a los nombres originales de las clases ayuda en el reporte.
reporte = classification_report(Y_test, Y_pred, target_names=target_classes, zero_division=0)
print("\n--- Reporte de Clasificación ---\n")
print(reporte)

# 3. Mostrar la Matriz de Confusión (para ver errores por clase)
conf_matrix = confusion_matrix(Y_test, Y_pred)
print("\n--- Matriz de Confusión ---\n")
# Imprimir solo la forma para evitar una matriz gigante
print(f"Forma de la Matriz de Confusión: {conf_matrix.shape}")


✨ Precisión del Modelo (Accuracy): 0.9683

--- Reporte de Clasificación ---

                                                  precision    recall  f1-score   support

      ACTIVIDADES ACUATICAS - ALTO DE LOS SUEÑOS       1.00      1.00      1.00       815
ACTIVIDADES SEDE PALMERAS DEPORT. Y ACTIV FISICA       0.46      0.21      0.29        29
                       ACTIVIDADES SUB ACUATICAS       1.00      0.20      0.33        10
                                         AJEDREZ       1.00      0.80      0.89        20
                                        ARQUERÍA       0.96      0.71      0.81        31
                                       ATLETISMO       0.94      1.00      0.97       118
                                       BADMINTON       1.00      0.19      0.32        21
                                      BALONCESTO       0.98      0.92      0.95       207
                                         BEISBOL       1.00      0.98      0.99        45
                     

## Conclusión

En este notebook, hemos construido con éxito un clasificador Naive Bayes para predecir el desempeño docente. Hemos cubierto el ciclo de vida completo de un proyecto de Machine Learning: desde la carga y preprocesamiento de los datos, pasando por el entrenamiento y la evaluación del modelo, hasta su aplicación para hacer nuevas predicciones. Este es un ejemplo excelente de cómo una técnica clásica y eficiente como Naive Bayes puede ser aplicada para extraer valor de los datos.

In [ ]:
# Verificar dependencias instaladas
import pkg_resources
import subprocess

required_packages = {
    'pandas': 'pandas',
    'scikit-learn': 'sklearn',
    'openpyxl': 'openpyxl',  # Necesario para leer archivos Excel
    'numpy': 'numpy'
}

def check_package(package_name):
    try:
        pkg_resources.require(package_name)
        return True
    except pkg_resources.DistributionNotFound:
        return False

print("Estado de las dependencias necesarias:")
missing_packages = []
for package, import_name in required_packages.items():
    if check_package(package):
        print(f"✅ {package} - Instalado")
    else:
        print(f"❌ {package} - No instalado")
        missing_packages.append(package)

if missing_packages:
    print("\nComando para instalar las dependencias faltantes:")
    print(f"pip install {' '.join(missing_packages)}")
else:
    print("\n✨ ¡Todas las dependencias necesarias están instaladas!")